# TME deconvolution - exons

In [1]:
source(here::here("scripts/init.R"))

Loading methylayer


Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following object is masked from ‘package:tidyr’:

    matches




In [2]:
prom_avg_meth <- get_promoter_avg_meth()

In [3]:
expr_mat <- get_gene_expression_mat() %>% expr_intervs_to_mat()

In [4]:
exon_meth <- get_exon_avg_meth()

### ER+ samples

In [5]:
ER_positive_norm_meth <- deconv_TME_metabric(prom_avg_meth, exon_meth, expr_mat, ER_positive_samples, k = 30) %cache_rds% here("data/ER_positive_norm_exon_meth.rds")

min_sd: 0.0965215274820842

min_expr: 6.00725469829817

calculating em-cross

# of samples: 1108

expression (columns): 16806

methylation (rows): 3946

2701 rows had at least one cor > 0.25

5879 columns had at least one cor > 0.25

5879 columns did not have more than 0 Na's 

2701 rows did not have more than 0 Na's 

clustering em-cross

normalizing methylation

immune module: 2

cafs module: 8

calculating TME methylation scores



### ER- samples

In [6]:
ER_negative_norm_meth <- deconv_TME_metabric(prom_avg_meth, exon_meth, expr_mat, ER_negative_samples, k = 15) %cache_rds% here("data/ER_negative_norm_exon_meth.rds")

min_sd: 0.0963502916561555

min_expr: 6.00725469829817

calculating em-cross

# of samples: 310

expression (columns): 16806

methylation (rows): 3684

3525 rows had at least one cor > 0.25

11054 columns had at least one cor > 0.25

11054 columns did not have more than 0 Na's 

3525 rows did not have more than 0 Na's 

clustering em-cross

normalizing methylation

immune module: 16

cafs module: 18

calculating TME methylation scores



### Normal samples

In [7]:
normal_norm_meth <- deconv_TME_metabric(prom_avg_meth, exon_meth, expr_mat, normal_samples, k = 15) %cache_rds% here("data/normal_norm_exon_meth.rds")

min_sd: 0.0656269964434713

min_expr: 6.00725469829817

calculating em-cross

# of samples: 92

expression (columns): 16806

methylation (rows): 2636

2636 rows had at least one cor > 0.25

16806 columns had at least one cor > 0.25

16806 columns did not have more than 0 Na's 

2636 rows did not have more than 0 Na's 

clustering em-cross

normalizing methylation

immune module: 3

cafs module: 11

calculating TME methylation scores



## Merge all normalized methylation

In [8]:
ER_positive_norm_meth$norm_meth[1:5, 1:5]

,MB_0006,MB_0028,MB_0046,MB_0050,MB_0053
chr1_136952_139281,NA,-0.005590510,-0.067832143,0.054737150,-0.00648710
chr1_761585_762902,-0.001658301,-0.007599587,-0.009553801,-0.008052132,NA
chr1_763063_763155,-0.026212291,-0.065095632,0.165035064,-0.025467726,-0.06592218
chr1_847324_850328,-0.048563238,-0.004047826,0.075698988,-0.047522828,0.17408909
chr1_861120_861180,-0.030213700,-0.037696280,-0.004541924,0.002994665,0.07373253


In [9]:
all_norm_meth <- cbind(ER_positive_norm_meth$norm_meth, ER_negative_norm_meth$norm_meth, normal_norm_meth$norm_meth) %>% mat_to_intervs()

In [10]:
all_norm_meth[1:5, 1:5]

,chrom,start,end,MB_0006,MB_0028
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1,136952,139281,NA,-0.005590510
2,chr1,761585,762902,-0.001658301,-0.007599587
3,chr1,763063,763155,-0.026212291,-0.065095632
4,chr1,847324,850328,-0.048563238,-0.004047826
5,chr1,861120,861180,-0.030213700,-0.037696280


In [11]:
dim(all_norm_meth)

[1] 29512  1513

In [12]:
fwrite(all_norm_meth, here("data/all_norm_exon_meth.csv"))

In [50]:
feats_df <- fread(here("data/epigenomic_features.tsv"))
exon_norm_meth <- all_norm_meth %>% as_tibble()
exon_cors <- bind_rows(
        epi_features_loci_cors(exon_norm_meth, feats_df, ER_positive_samples) %>% mutate(ER = "ER+"), 
        epi_features_loci_cors(exon_norm_meth, feats_df, ER_negative_samples) %>% mutate(ER = "ER-"), 
        epi_features_loci_cors(exon_norm_meth, feats_df, normal_samples) %>% mutate(ER = "normal")
    ) %>% 
    select(chrom:end, ER, everything()) %fcache_df%
    here("data/features_exon_cors.tsv") %>% 
    as_tibble() 
loci_annot_exons <- exon_cors %>% 
    filter(ER == "ER+") %>% 
    select(-ER) %>% 
    annotate_loci() %cache_df% 
    here("data/exon_annot_epigenomic_features.tsv") %>% 
    as_tibble()    